In [1]:
# set up the model
setwd('/lustre/scratch117/cellgen/team297/kt16/newcastle_covid')
library(dplyr)

bcell.df <- read.csv("h5ad/combined_dec_bcells_obs.csv")

covid.meta <- bcell.df[,c('sample_id', 'patient_id', 'Status_on_day_collection_summary', 'Age', 'Sex', 'Collection_Day')] %>% distinct()
rownames(covid.meta) <- covid.meta$sample_id

bcell.meta <- covid.meta[!covid.meta$Status_on_day_collection_summary %in% c("Non_covid", "LPS", 'Healthy'),]
bcell.meta$OrderedSeverity <- ordered(bcell.meta$Status_on_day_collection_summary, levels=c("Asymptomatic", "Mild", "Moderate", "Severe", "Critical"))

bcell.model <- model.matrix(~ Sex + Age + OrderedSeverity, data=bcell.meta[bcell.meta$Collection_Day %in% c("D0"), ])


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
# count cells
cell.freq.tab <- t(table(bcell.df$sample_id[bcell.df$Collection_Day %in% c("D0") &
                                                        !bcell.df$Status_on_day_collection_summary %in% c("LPS", "Non_covid", 'Healthy')],
                         bcell.df$celltype_B[bcell.df$Collection_Day %in% c("D0") &
                                                        !bcell.df$Status_on_day_collection_summary %in% c("LPS", "Non_covid", 'Healthy')]))
cell.freq.tab <- cell.freq.tab[,colSums(cell.freq.tab) != 0]
bcell.model <- bcell.model[colnames(cell.freq.tab), ]

n.cell.vecc <- table(bcell.df$sample_id)

library(edgeR)
bcell.dge <- DGEList(cell.freq.tab, lib.size=log(n.cell.vecc[colnames(cell.freq.tab)]))

Loading required package: limma



In [3]:
#estimate dispersions and fit model
bcell.dge <- estimateDisp(bcell.dge, design=bcell.model)
bcell.linear.fit <- glmQLFit(bcell.dge, bcell.model, robust=TRUE)

In [4]:
# linear changes
bcell.res <- as.data.frame(topTags(glmQLFTest(bcell.linear.fit, coef=4), sort.by='none', n=Inf))
bcell.res$CellType <- rownames(bcell.res)
bcell.res$Sig <- as.numeric(bcell.res$FDR < 0.1 & bcell.res$PValue < 0.05)
bcell.res$Diff <- sign(bcell.res$logFC)
bcell.res$Diff[bcell.res$FDR >= 0.1 | bcell.res$PValue >= 0.05] <- 0
bcell.res

,logFC,logCPM,F,PValue,FDR,CellType,Sig,Diff
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
B_exhausted,0.150996407,15.53285,2.235211e-01,0.637485703,0.81962448,B_exhausted,0,0
B_immature,-0.137819622,16.20341,2.676712e-01,0.606135453,0.81962448,B_immature,0,0
B_naive,-0.075732684,19.16972,5.720534e-01,0.451369539,0.81246517,B_naive,0,0
B_non-switched_memory,0.001598324,15.32163,3.439785e-05,0.995333113,0.99533311,B_non-switched_memory,0,0
B_switched_memory,-0.201763822,16.58476,8.329594e-01,0.363795033,0.81246517,B_switched_memory,0,0
Plasma_cell_IgA,0.031776845,15.53893,8.833205e-03,0.925324612,0.99533311,Plasma_cell_IgA,0,0
Plasma_cell_IgG,0.702165753,15.63913,4.005067e+00,0.048299307,0.14489792,Plasma_cell_IgG,0,0
Plasma_cell_IgM,1.152909165,13.77362,5.523214e+00,0.020895361,0.09402912,Plasma_cell_IgM,1,1
Plasmablast,1.145440423,15.78431,7.525513e+00,0.007307719,0.06576947,Plasmablast,1,1


In [5]:
# Quadratic changes.
bcell.quad.res <- as.data.frame(topTags(glmQLFTest(bcell.linear.fit, coef=5), sort.by='none', n=Inf))
bcell.quad.res$CellType <- rownames(bcell.quad.res)
bcell.quad.res$Sig <- as.numeric(bcell.quad.res$FDR < 0.1 & bcell.quad.res$PValue < 0.05)
bcell.quad.res$Diff <- sign(bcell.quad.res$logFC)
bcell.quad.res$Diff[bcell.quad.res$FDR >= 0.1 | bcell.quad.res$PValue >= 0.05] <- 0
bcell.quad.res

,logFC,logCPM,F,PValue,FDR,CellType,Sig,Diff
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
B_exhausted,-0.045572430,15.53285,0.025176007,8.742758e-01,9.802455e-01,B_exhausted,0,0
B_immature,0.147918339,16.20341,0.394644308,5.314190e-01,6.832530e-01,B_immature,0,0
B_naive,0.147298655,19.16972,2.743819985,1.010257e-01,2.273078e-01,B_naive,0,0
B_non-switched_memory,-0.005958338,15.32163,0.000616447,9.802455e-01,9.802455e-01,B_non-switched_memory,0,0
B_switched_memory,0.154785990,16.58476,0.612720312,4.357684e-01,6.536526e-01,B_switched_memory,0,0
Plasma_cell_IgA,-0.402075312,15.53893,1.663282190,2.003818e-01,3.606872e-01,Plasma_cell_IgA,0,0
Plasma_cell_IgG,-1.015626469,15.63913,9.926193106,2.196571e-03,9.884569e-03,Plasma_cell_IgG,1,-1
Plasma_cell_IgM,-2.071181486,13.77362,24.084565899,3.952369e-06,3.557132e-05,Plasma_cell_IgM,1,-1
Plasmablast,-0.966937241,15.78431,6.464662574,1.266838e-02,3.800513e-02,Plasmablast,1,-1
